In [36]:
import pandas as pd
import sqlite3
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time

## Selenium Scraping

In [31]:
url = 'https://www.imdb.com/search/title/?title_type=feature'

driver = webdriver.Chrome()

driver.get(url)

time.sleep(2)


In [130]:
for i in range(20):
    l = driver.find_element(By.CSS_SELECTOR, ".cMGrFN button")
    driver.execute_script("window.scrollTo(0, arguments[0])", l.location["y"]-200)
    time.sleep(1)
    driver.execute_script('arguments[0].removeAttribute("disabled");', l)
    driver.execute_script("arguments[0].setAttribute('value',true)",l)
    time.sleep(1)
    l.click()
    time.sleep(2)

In [132]:
html_text = driver.page_source
time.sleep(2) 
driver.quit()

## ETL

In [348]:
def extract() -> dict:
    title = []
    duration = []
    metascore = []
    director = []
    stars = []
    quick_def = [] 
    age_rating = [] 
    year = []
    score = []
    # voters = []

    soup = BeautifulSoup(html_text, "lxml")
    
    all_films = soup.find("ul", class_="ipc-metadata-list")
    
    for j in all_films.find_all("li"):
        t_ = ""
        y_ = 0
        d_ = 0
        a_ = ""
        try:
            t_ = (" ".join(j.find("h3", class_="ipc-title__text").text.strip().split(" ")[1:]))
            sub = j.find("div", "sc-300a8231-6 dBUjvq dli-title-metadata")
            y_ = (sub.find_all("span")[0].text.strip())
            d_ = (sub.find_all("span")[1].text.strip())
            a_ = (sub.find_all("span")[2].text.strip())
        except:
            continue
        title.append(t_)
        year.append(y_)
        duration.append(d_)
        age_rating.append(a_)
        meta_ = 0
        try:
            meta_ = (j.find_all("span", class_="sc-300a8231-9 fNNdeo")[0].find("span", class_="bXIOoL").text.strip())
            metascore.append(int(meta_))
        except:
            metascore.append("none")
        
        # driver = webdriver.Chrome()
        # driver.get("https://www.imdb.com" + j.find("a", class_="ipc-title-link-wrapper")["href"])
        
        # time.sleep(1)
        # html_ = driver.page_source
        # driver.quit()
        
        # site_details = BeautifulSoup(html_, "lxml")
        # print("https://www.imdb.com" + j.find("a", class_="ipc-title-link-wrapper")["href"])
        # print(site_details)
        # if site_details.find("title").text.strip() == "403 Forbidden":
        #     metascore.append("none")
        #     score.append("none")
        #     director.append("none")
        #     stars.append('none')
        #     continue

        # try:
        #     reviews = site_details.find("ul", class_="ipc-inline-list").find_all("li")[2].find("span", class_="metacritic-score-box").text.strip()
        #     metascore.append(int(reviews))
        # except:
        #     metascore.append("none")
        
        # try:
        #     star_rate = site_details.find("span", class_="sc-d541859f-1 imUuxf").text.strip()
        #     score.append(float(star_rate))
        # except:
        #     score.append("none")

        # dir_wr_strs = site_details.find("div", class_="sc-70a366cc-2 bscNnP").find("ul", class_="ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt").find_all("li")
        # try:
        #     dir = dir_wr_strs[0].find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text.strip()
        #     director.append(dir)
        # except:
        #     director.append("none")
        # try:
        #     star = dir_wr_strs[2].find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text.strip()
        #     stars.append(star)
        # except:
        #     stars.append('none')
            
        # try:
        #     quick_def.append(j.find("div", "sc-3ac15c8d-0 hRUoSB").text.strip())
        # except:
        #     quick_def.append("none")
    
    return {
    "title": title,
    "duration": duration,
    "metascore": metascore,
    # "director": director,
    # "stars": stars,
    # "quick_def": quick_def,
    "age_rating": age_rating,
    "year": year,
    # "score": score,
    }

    


In [366]:
def transform(d: dict):
    return pd.DataFrame(d)

In [378]:
def load(df: pd.DataFrame):
    conn = sqlite3.connect('movies.db')
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS movies (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        duration TEXT,
        metascore float,
        age_rating TEXT,
        year TEXT
    );
    """)
    conn.commit()
    df.to_sql("movies", conn, if_exists="append", index=False)
    conn.commit()
    conn.close()

In [380]:
[len(i) for i in data.values()]

[1243, 1243, 1243, 1243, 1243]

In [382]:
data = extract()
df = transform(data)
load(df)